In [1]:
import tarfile, lzma
import pandas as pd
import nltk.data
import warnings
import nltk.tokenize
from nltk.grammar import DependencyGrammar
from nltk.parse import (
    DependencyGraph,
    ProjectiveDependencyParser,
    NonprojectiveDependencyParser,
)
from zss import simple_distance, Node
# import spacy
from collections import defaultdict, Counter
import numpy as np
import warnings
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings('ignore')

In [2]:
arch =  tarfile.open('Russian-annotated-conll17.tar')

In [3]:
def iterate_archive(arch):
    for file in arch:
        if 'wiki' not in file.name:
            continue
        if not file.isfile():
            continue
        file = arch.extractfile(file)
        try:
            reader = lzma.open(file)
        except:
            continue
        par = []
        sent = None
        tree = []
        first = reader.readline().decode('utf8')
        
        if first == '# newpar\n':
            pars = True
        else:
            pars = False
            sent = first
        
        
        for line in reader:
            text = line.decode('utf8')
            if text == '# newpar\n':
                if pars:
                    if sent and tree:
                        par.append((sent, ''.join(tree)))
                
                    yield par
                else:
                    yield [(sent, ''.join(tree))]
                    pars = True
                
                par = []
                sent, tree = None, []
            
            elif text == '\n':
                if pars and sent and tree:
                    par.append((sent, ''.join(tree)))
                
                elif not pars and sent and tree:
                    yield [(sent, ''.join(tree))]
                
                sent, tree = None, []
            
            elif text.startswith('#'):
                if sent:
                    print(sent, tree, text)
                    print('Error, line with # in the wrong place!')
                    raise KeyError
                sent = text
            
            else:
                tree.append(text)

def get_subtree(nodes, node):
    if not nodes[node]['deps']:
        return [node]
    else:
        
        
        return [node] + [get_subtree(nodes, dep) for rel in nodes[node]['deps'] if rel != 'punct'
                         for dep in nodes[node]['deps'][rel]]
# flatten = lambda l: [item for sublist in l for item in sublist]
def flatten(l):
    flat = []
    for el in l:
        if not isinstance(el, list):
            flat.append(el)
        else:
            flat += flatten(el)
    return flat

def get_predicate(d):
    subj = []
    main_subj = None
    obj = []
    main_obj = None
    verbs = []
    nodes = d.nodes
    for node in nodes:
        if nodes[node]['ctag'] == 'VERB':
            verb = [node]
            if 'aux' in nodes[node]['deps']:
                verb += flatten([get_subtree(nodes, n) for n in nodes[node]['deps']['aux']])
            elif 'aux:pass' in nodes[node]['deps']:
                verb += flatten([get_subtree(nodes, n) for n in nodes[node]['deps']['aux:pass']])
            
            verbs.append(verb)
    
    if not verbs:
#         print('BAD!')
        return None
    
    predicates = []
    for verb in verbs:
        if ('obj' in nodes[verb[0]]['deps'] or 'obl' in nodes[verb[0]]['deps']) \
            and ('nsubj' in nodes[verb[0]]['deps'] or 'nsubj:pass' in nodes[verb[0]]['deps']):
            
            main_subj = nodes[verb[0]]['deps'].get('nsubj', None) or nodes[verb[0]]['deps']['nsubj:pass']
            main_subj = main_subj[0]
            main_objs = nodes[verb[0]]['deps'].get('obj', []) + nodes[verb[0]]['deps'].get('obl', [])
    
            subj = sorted(flatten(get_subtree(nodes, main_subj)))
            objs = [sorted(flatten(get_subtree(nodes, main_obj))) for main_obj in main_objs]
    
            subj_words = [nodes[node]['word']+'_'+nodes[node]['ctag'] \
                          if node > 1 else '<start>_'+nodes[node]['word']+'_'+nodes[node]['ctag'] \
                          for node in subj]
            obj_words = []
            for obj in objs:
                ow = []
                for node in obj:
                    ow.append(nodes[node]['word']+'_'+nodes[node]['ctag'])
                obj_words.append(ow)
            
            verb_words = [nodes[node]['word']+'_'+nodes[node]['ctag'] for node in sorted(verb)]
            pred = []
            pred.append(' '.join(subj_words))
            pred.append(' '.join(verb_words))
            for ow in sorted(obj_words, key=lambda x: x[0]):
                pred.append(' '.join(ow))
            
            predicates.append(pred)
    
    return predicates

def get_predicate_pattern(d):
    subj = []
    main_subj = None
    obj = []
    main_obj = None
    verbs = []
    nodes = d.nodes
    for node in nodes:
        if nodes[node]['ctag'] == 'VERB':
            verb = [node]
            if 'aux' in nodes[node]['deps']:
                verb += flatten([get_subtree(nodes, n) for n in nodes[node]['deps']['aux']])
            elif 'aux:pass' in nodes[node]['deps']:
                verb += flatten([get_subtree(nodes, n) for n in nodes[node]['deps']['aux:pass']])
            
            verbs.append(verb)
    
    if not verbs:
#         print('BAD!')
        return None
    
    predicates = []
    for verb in verbs:
        if ('obj' in nodes[verb[0]]['deps'] or 'obl' in nodes[verb[0]]['deps']) \
            and ('nsubj' in nodes[verb[0]]['deps'] or 'nsubj:pass' in nodes[verb[0]]['deps']):
            
            main_subj = nodes[verb[0]]['deps'].get('nsubj', None) or nodes[verb[0]]['deps']['nsubj:pass']
            main_subj = main_subj[0]
            main_objs = nodes[verb[0]]['deps'].get('obj', []) + nodes[verb[0]]['deps'].get('obl', [])
    
            subj = sorted(flatten(get_subtree(nodes, main_subj)))
            objs = [sorted(flatten(get_subtree(nodes, main_obj))) for main_obj in main_objs]
    
            subj_words = [nodes[node]['word']+'_'+nodes[node]['ctag'] \
                          if node > 1 else '<start>_'+nodes[node]['word']+'_'+nodes[node]['ctag'] \
                          for node in subj]
            obj_words = []
            obj_i = []
            for obj in objs:
                ow = []
                for node in obj:
                    ow.append(nodes[node]['word']+'_'+nodes[node]['ctag']+'_'+nodes[node]['rel'])
                    obj_i.append(node)
                obj_words.append(ow)
                    
            
            obj_i = sorted(obj_i)
            pattern_i = []
            if subj[0] > obj_i[0]:
                pattern_i += list(range(subj[-1]+1, obj_i[0]))
            else:
                pattern_i += list(range(subj[-1]+1, obj_i[0]))
            if not pattern_i:
                continue
            verb_words = []
            for node in pattern_i:
                w = nodes[node]['word'] + '_'+nodes[node]['ctag']+'_'+nodes[node]['rel']
                if node in verb:
                    w += '_CORE'
                verb_words.append(w)
            
            pred = []
            pred.append(' '.join(subj_words))
            pred.append(' '.join(verb_words))
            for ow in sorted(obj_words, key=lambda x: x[0]):
                pred.append(' '.join(ow))
            
            predicates.append(pred)
    
    return predicates

In [358]:
[] + list(range(1, 3))

[1, 2]

In [106]:
a = [f for f in arch]

In [108]:
b = a[1]

In [7]:
j

283966

In [11]:
i = iterate_archive(arch)

In [12]:
fs = open('sentences_100k_wiki.txt', 'w')
# ftr = open('trees_50000.txt', 'w')

for j, t in enumerate(i):
    
    if j > 300000:
        break
    for s in t:
        
        sent, tree = s
        if not sent or len(sent) > 1000:
            continue
        
        fs.write(sent.replace('# text = ', ''))
#         ftr.write(tree + '\n')
fs.close()
# ftr.close()

In [71]:
next(i)[0][0].replace('# text = ', '')

'Есть ли там ресторан или кафе?\n'

In [89]:
next(i)

[('# text = Автошкола “ВИРАЖ” работает на основании таких нормативных документов, как: лицензия Министерства образования Украины (Серия АЕ No 527752 от 15.12.2014 г.) и сертификат Министерства внутренних дел (Серия ДДАІ No 003029 от 03.07.2015 г.).\n',
  '1\tАвтошкола\tАвтошкол\tNOUN\t_\tAnimacy=Inan|Case=Nom|Gender=Fem|Number=Sing\t3\tnsubj\t_\t_\n2\t“ВИРАЖ”\t“ВИРАЖ”\tPROPN\t_\tAnimacy=Anim|Case=Nom|Gender=Masc|Number=Sing\t1\tappos\t_\t_\n3\tработает\tработать\tVERB\t_\tAspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act\t0\troot\t_\t_\n4\tна\tна\tADP\t_\t_\t5\tcase\t_\t_\n5\tосновании\tоснование\tNOUN\t_\tAnimacy=Inan|Case=Loc|Gender=Neut|Number=Sing\t3\tobl\t_\t_\n6\tтаких\tтакой\tDET\t_\tCase=Gen|Number=Plur\t8\tamod\t_\t_\n7\tнормативных\tнормативный\tADJ\t_\tCase=Gen|Degree=Pos|Number=Plur\t8\tamod\t_\t_\n8\tдокументов\tдокумент\tNOUN\t_\tAnimacy=Inan|Case=Gen|Gender=Masc|Number=Plur\t5\tnmod\t_\tSpaceAfter=No\n9\t,\t,\tPUNCT\t_\t_\t8\tpunct\t_\t_\n10\tкак

In [355]:
f = open('predicates_pos.txt', 'w')
for par in iterate_archive(arch):
    for sent, tree in par:
        try:
            d = DependencyGraph(tree.split('\n', maxsplit=1)[1])
        except:
            continue
        pr = get_predicate(d)
        if not pr:continue
        for p in pr:
            f.write('\t'.join(p) + '\n')
            
f.close()

KeyboardInterrupt: 

In [448]:
f = open('predicates_pattern.txt', 'w')
for par in iterate_archive(arch):
    for sent, tree in par:
        try:
            d = DependencyGraph(tree.split('\n', maxsplit=1)[1])
        except:
            continue
        pr = get_predicate_pattern(d)
        if not pr:continue
        for p in pr:
            f.write('\t'.join(p) + '\n')
            
f.close()

KeyboardInterrupt: 

In [449]:
f.close()

In [356]:
f.close()

## Близость предикатов по embeddingам

In [12]:
import sentencepiece as spm
sp = spm.SentencePieceProcessor()
sp.Load("ru.wiki.bpe.op200000.model")
# sp.EncodeAsPieces("This is a test")

True

In [13]:
sp.EncodeAsPieces("применяется")

['▁применяется']

In [14]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format("ru.wiki.bpe.op200000.d300.w2v.bin", binary=True)

In [182]:
np.concatenate([np.mean(model[sp.EncodeAsPieces("применяется на практике")], axis=0), 
          np.mean(model[sp.EncodeAsPieces("применяется на практике")], axis=0)]).shape

(600,)

In [25]:
sp.EncodeAsPieces(p.lower())

['▁', 'ב', 'пере', 'ходит']

In [186]:
a = np.zeros((100,))

In [185]:
model.

In [243]:
predicates = open('predicates.txt').read().splitlines()
X = np.zeros((len(predicates), 300))
# X_subj = np.zeros((len(predicates), 300))
for i, pred in enumerate(predicates):
    pl = pred.split('\t')
    
    if len(pl) > 5:
        continue
    
    nsubj, p, *objs = pl
    
#     pieces = [x for x in sp.EncodeAsPieces(p.lower()) if x in model]
#     if not pieces:
#         continue
#     vecs = model[pieces]
#     vec = np.mean(vecs, axis=0)
#     X[i] = vec
    nsubj_pieces = [x for x in sp.EncodeAsPieces(nsubj.lower()) if x in model]
    nsubj_vec = np.mean(model[nsubj_pieces], axis=0) if nsubj_pieces else np.zeros((300,))
    
#     p_pieces = [x for x in sp.EncodeAsPieces(p.lower()) if x in model]
#     p_vec = np.mean(model[p_pieces], axis=0) if p_pieces else np.zeros((300,))
    
#     objs_pieces = [[x for x in sp.EncodeAsPieces(obj.lower()) if x in model] for obj in objs]
#     obj_vec = [np.mean(model[piece], axis=0) if piece else np.zeros((300,)) for piece in objs_pieces] 
    
#     vec = [nsubj_vec, p_vec] + obj_vec + [np.zeros((300,)) for i in range(3 - len(objs))]
    
#     vec = np.concatenate(p_vec)
    
    X[i] = nsubj_vec
    

In [191]:
model[[x for x in sp.EncodeAsPieces(nsubj.lower()) if x in model]].shape

(1, 300)

In [226]:
X.shape

(144660, 300)

In [195]:
[x for x in sp.EncodeAsPieces(nsubj.lower()) if x in model]

[]

In [29]:
from sklearn.cluster import MiniBatchKMeans

In [278]:
km = MiniBatchKMeans(10000, n_init=1, batch_size=20000, verbose=True, init_size=30000,  max_no_improvement=150)

In [227]:
km_s = MiniBatchKMeans(10000, n_init=1, batch_size=20000, verbose=True)

In [280]:
# km_s.fit(X)

In [281]:
km.fit(X)

Init 1/1 with method: k-means++
Inertia for init 1/1: 75306.466880
Minibatch iteration 1/800: mean batch inertia: 3.778345, ewa inertia: 3.778345 
Minibatch iteration 2/800: mean batch inertia: 3.602757, ewa inertia: 3.729794 
Minibatch iteration 3/800: mean batch inertia: 3.440036, ewa inertia: 3.649673 
Minibatch iteration 4/800: mean batch inertia: 3.434478, ewa inertia: 3.590170 
Minibatch iteration 5/800: mean batch inertia: 3.275418, ewa inertia: 3.503138 
Minibatch iteration 6/800: mean batch inertia: 3.297392, ewa inertia: 3.446248 
Minibatch iteration 7/800: mean batch inertia: 3.276970, ewa inertia: 3.399441 
Minibatch iteration 8/800: mean batch inertia: 3.197869, ewa inertia: 3.343705 
Minibatch iteration 9/800: mean batch inertia: 3.219714, ewa inertia: 3.309420 
[MiniBatchKMeans] Reassigning 9752 cluster centers.
Minibatch iteration 10/800: mean batch inertia: 3.159101, ewa inertia: 3.267856 
Minibatch iteration 11/800: mean batch inertia: 5.277688, ewa inertia: 3.823592 

Minibatch iteration 101/800: mean batch inertia: 4.291330, ewa inertia: 4.273622 
Minibatch iteration 102/800: mean batch inertia: 4.250618, ewa inertia: 4.267261 
Minibatch iteration 103/800: mean batch inertia: 4.296551, ewa inertia: 4.275360 
Minibatch iteration 104/800: mean batch inertia: 4.272898, ewa inertia: 4.274679 
Minibatch iteration 105/800: mean batch inertia: 4.218040, ewa inertia: 4.259018 
Minibatch iteration 106/800: mean batch inertia: 4.193293, ewa inertia: 4.240844 
Minibatch iteration 107/800: mean batch inertia: 4.271274, ewa inertia: 4.249258 
Minibatch iteration 108/800: mean batch inertia: 4.246664, ewa inertia: 4.248541 
Minibatch iteration 109/800: mean batch inertia: 4.186650, ewa inertia: 4.231428 
Minibatch iteration 110/800: mean batch inertia: 4.212890, ewa inertia: 4.226302 
Minibatch iteration 111/800: mean batch inertia: 4.202998, ewa inertia: 4.219858 
Minibatch iteration 112/800: mean batch inertia: 4.217906, ewa inertia: 4.219318 
Minibatch iterat

MiniBatchKMeans(batch_size=20000, compute_labels=True, init='k-means++',
        init_size=30000, max_iter=100, max_no_improvement=150,
        n_clusters=10000, n_init=1, random_state=None,
        reassignment_ratio=0.01, tol=0.0, verbose=True)

In [270]:
km.

MiniBatchKMeans(batch_size=20000, compute_labels=True, init='k-means++',
        init_size=30000, max_iter=100, max_no_improvement=10,
        n_clusters=10000, n_init=1, random_state=None,
        reassignment_ratio=0.01, tol=0.0, verbose=True)

In [275]:
labels.shape

(20000,)

In [284]:
labels = km.labels_

In [47]:
labels_s = km_s.labels_

In [285]:
clusters = defaultdict(list)
for i in range(len(predicates)):
    clusters[labels[i]].append(predicates[i])

In [48]:
clusters = defaultdict(list)
for i in range(len(predicates)):
    clusters[labels_s[i]].append(predicates[i])

In [237]:
m = cosine_similarity(X[9].reshape(1, -1), X)

In [240]:
m.argsort()[0][::-1]

IndexError: index 100076 is out of bounds for axis 0 with size 1

In [241]:
predicates[100076]

'Борис Львович Коваленко\tродился\tв с . Хотуничи Щорского района Черниговской области в семье железнодорожника .'

In [54]:
ml = Counter()
for p in predicates:
    ml.update([len(p.split('\t'))])

In [55]:
ml

Counter({3: 82381, 4: 35547, 5: 6518, 6: 862, 7: 112, 8: 14, 9: 4})

In [286]:
f = open('clusters_subj.txt', 'w')
for cl in clusters:
    f.write('### {} ###\n\n'.format(cl))
    pred_by_length = defaultdict(list)
    
#     for p in clusters[cl]:
#         if len(p.split('\t')) < 3:
#             continue
#         pred_by_length[len(p.split('\t')) - 2].append(p)
        
#     for l in pred_by_length:
#         f.write('\n\n@@@ {} @@@\n\n'.format(l))
#         for p in pred_by_length[l]:
    for p in clusters[cl]:
        f.write(p.split('\t')[0] + '\n')
    
    f.write('\n\n')
f.close()

In [31]:
def count_verb_rels(sents):
    verbs = defaultdict(set)
    for sent in sents:
        try:
            d = DependencyGraph(sent.split('\n', maxsplit=1)[1])
        except (KeyError, ValueError, AssertionError):
            continue
        nodes = [node for node in d.nodes if d.nodes[node]['ctag'] == 'VERB']
        
        for node in nodes:
            v = d.nodes[node]['word'].lower()
            subs = []
            for rel in d.nodes[node]['deps']:
#                 if rel not in ['obl', 'obj']:
#                     continue
                if rel not in ['aux', 'aux:pass']:
                    continue
                
                for s in d.nodes[node]['deps'][rel]:
                    sub = sorted(flatten(get_subtree(d.nodes, s)))
#                     subs = [(d.nodes[n]['rel'], d.nodes[n]['word']) for n in sub]
                    subs.append(' '.join([d.nodes[n]['word'] for n in sub]))
#                         sw = (d.nodes[s]['rel'], d.nodes[s]['word'].lower())
#                             verbs[v].update([sw])
            if subs:
                verbs[v].update(tuple(subs))
    return dict(verbs)

In [66]:
flatten(get_subtree(d.nodes, 2))

[2, 3, 4]

In [162]:
verbs = count_verb_rels(sents[:100000])

In [167]:
vs = list(verbs.keys())

In [168]:
vs[:10]

['объединена',
 'уступать',
 'проводиться',
 'скрыта',
 'проголосовало',
 'отпущен',
 'посажены',
 'потерян',
 'переделаны',
 'вынуждено']

In [170]:
set(verbs['объединена'])

{'была'}

In [173]:
[] + [1, 2,3]

[1, 2, 3]

In [2]:
import gensim

In [3]:
from gensim.corpora import WikiCorpus, MmCorpus
from gensim.utils import tokenize, lemmatize
import gensim


In [6]:
# wiki = WikiCorpus('ruwiki-latest-pages-articles.xml.bz2', lemmatize=True) # create word->word_id mapping, takes almost 8h
MmCorpus.serialize('wiki_ru_vocab.mm', wiki) # another 8h, creates a file in MatrixMarket format and mapping

In [16]:
f = open('wiki_texts.txt', 'w')
i = 0
for text in wiki.get_texts():
    f.write(' '.join([x.decode('utf8').rstrip('_') for x in text]) + '\n')
    i += 1
    if not i % 10000:
        print('Processed - ', i)

Processed -  10000
Processed -  20000
Processed -  30000
Processed -  40000
Processed -  50000
Processed -  60000
Processed -  70000
Processed -  80000
Processed -  90000
Processed -  100000
Processed -  110000
Processed -  120000
Processed -  130000
Processed -  140000
Processed -  150000
Processed -  160000
Processed -  170000
Processed -  180000
Processed -  190000
Processed -  200000
Processed -  210000
Processed -  220000
Processed -  230000
Processed -  240000
Processed -  250000
Processed -  260000
Processed -  270000
Processed -  280000
Processed -  290000
Processed -  300000
Processed -  310000
Processed -  320000
Processed -  330000
Processed -  340000
Processed -  350000
Processed -  360000
Processed -  370000
Processed -  380000
Processed -  390000
Processed -  400000
Processed -  410000
Processed -  420000
Processed -  430000
Processed -  440000
Processed -  450000
Processed -  460000
Processed -  470000
Processed -  480000
Processed -  490000
Processed -  500000
Processed

In [14]:
f.close()

In [15]:
i

142535

In [2]:
docs = gensim.models.doc2vec.TaggedLineDocument('wiki_texts.txt')

In [4]:
doc2vec = gensim.models.Doc2Vec(docs, vector_size=200, min_count=5, workers=3)

In [3]:
?gensim.models.Doc2Vec

In [21]:
doc2vec.docvecs.most_similar([doc2vec.infer_vector(t[0].split())])

[(684460, 0.910470187664032),
 (1182366, 0.9060593843460083),
 (1091552, 0.9052213430404663),
 (905271, 0.9049792885780334),
 (1013732, 0.9021373391151428),
 (1102873, 0.9021013975143433),
 (1032700, 0.9013933539390564),
 (1169996, 0.9012395143508911),
 (864957, 0.9008297324180603),
 (1142866, 0.9005771279335022)]

In [22]:
t = []

In [23]:
for i, text in enumerate(open('wiki_texts.txt')):
    if i == 684460:
        t.append(text)
        break

In [25]:
doc2vec.save('word2vec_200')

In [479]:
from nltk.corpus import stopwords
stops = set(stopwords.words('russian') + ['который', 'это', 'та', "которая", "всё", "многое", "которое"])

In [482]:
predicates = Counter()
examples = defaultdict(set)
for line in open('predicates_pattern.txt'):
    subj, predicate, *objs = line.split('\t')
    sws = [token.split('_')[0] for token in subj.split()]
    if not len(set([x.lower() for x in sws]) - stops):
        continue
    for obj in objs:
        tokens = obj.split()
        ws = [token.split('_')[0] for token in tokens]
        if not (set([x.lower() for x in ws]) - stops):
            continue
        rel = [token.split('_')[2] for token in tokens]
        pred_ws = [token.split('_')[0] for token in predicate.split()]
        if rel[0] in ['case', 'obl']:
            pred_ws.append(ws.pop(0))
            predicates.update([' '.join(pred_ws)])
        examples[' '.join(pred_ws)].add((' '.join(sws), ' '.join(ws)))

In [459]:
def examples_for_pred(pred):
    predicates = Counter()
    for line in open('predicates_pattern.txt'):
        subj, predicate, *objs = line.split('\t')
        p = ' '.join([token.split('_')[0] for token in predicate.split()])
        print(p)
        return False
        if pred == p:
            predicates.update([(' '.join([token.split('_')[0] for token in subj.split()]), 
                    ' '.join([token.split('_')[0] for token in objs[0].split()]))])
            
    return predicates.most_common()

In [483]:
examples['зависит от']

{('очень многое', 'людей от человеческого фактора'),
 ('эпигенетическое наследование ребенка',
  'объективных факторов стресса например количества дней проведенных без электричества'),
 ('толщина ленточного фундамента', 'используемого материала'),
 ('Решение', 'вероятности повторной госпитализации. &gt'),
 ('качество его сборки', 'выбора производителя и страны где он изготовлен'),
 ('биение сердца', 'количества кислорода в баллонах для дыхания'),
 ('львиная доля успеха',
  'верного расчета материалов и опыта не только строителей но и офисных менеджеров'),
 ('твое настроение', 'скорости'),
 ('цена которого', 'фронта предстоящих работ и применяемых средств'),
 ('ввод текста', 'контекста'),
 ('167 Наступление половой зрелости Наступление половой зрелости у крупного рогатого скота',
  'скороспелости животных'),
 ('уборка', 'наличия на двери соответствующей таблички'),
 ('состав которого',
  'концентрации кислоты например Fe + 4HNO3 = Fe( NO3)3 + NO + 2H2O.'),
 ('ее написание', 'того в каки

In [452]:
predicates.most_common()

[(', в', 5614),
 ('можете с', 5366),
 ('находится в', 5142),
 ('стабильно получают от', 3227),
 ('состоит из', 3222),
 ('относится к', 2714),
 ('зависит от', 2182),
 ('ищем по', 2053),
 ('включает в', 1866),
 ('награждён орденом', 1843),
 (', на', 1745),
 ('находятся в', 1737),
 ('находится на', 1705),
 ('приводит к', 1698),
 ('заключается в', 1589),
 (', размещенная пользователями на eKomok является конфиденциальной', 1535),
 ('состоят из', 1520),
 (') находится в', 1407),
 ('входит в', 1323),
 ('принадлежит к', 1282),
 ('связано с', 1169),
 ('является частью', 1120),
 (', с', 1119),
 ('происходит в', 1094),
 ('привело к', 1067),
 ('свяжется с', 1012),
 ('идет о', 977),
 ('состоит в', 970),
 ('сделаем от', 943),
 (') в', 923),
 ('говорит о', 912),
 ('не скрыл их', 868),
 ('не скрыл в', 868),
 (', по', 824),
 ('относятся к', 819),
 ('попал в', 785),
 ('принимала в', 733),
 ('( в', 720),
 ('поможет вам', 714),
 ('предназначен для', 703),
 ('участвовал в', 697),
 ('помогут вам', 679),
 (

In [438]:
preds = defaultdict(Counter)
sub_obj = defaultdict(Counter)
c = Counter()
for line in open('predicates_pattern.txt'):
    subj, predicate, *objs = line.split('\t')
    preds[predicate].update([subj])
    c.update([predicate])
    sub_obj[predicate].update([(subj, tuple(objs))])

KeyboardInterrupt: 

In [412]:
predicate

'и_PART_advmod дано_VERB_parataxis_CORE'

In [434]:
def verb_post(key, lookup):
    cnt = sub_obj[key].most_common()
    w_post = Counter()
    seqs_post = []
    df = defaultdict(set)
    for t, c in cnt:
        for obj in t[1]:
            tokens = obj.split()
            w_post[tokens[0].split('_')[0]] += c
            pos = ' '.join([token.split('_')[2] for token in tokens])
            seqs_post.append(pos)
            df[pos].add(' '.join([token.split('_')[0] for token in tokens]))
    return w_post.most_common(), seqs_post, df

In [435]:
w, seqs, d = verb_post('предлагает_VERB_root_CORE', sub_obj)

In [442]:
sub_obj['является'].most_common()

[(('информация', ('конфиденциальной\n',)), 4871),
 (('блокировка', ('в таком случае', 'полной\n')), 614),
 (('лицензия', ('на него', 'свободной\n')), 450),
 (('отель', ('частью сети отелей\n',)), 268),
 (('рейсовСравнить',
   ('собственностью компании BravoNext SA с головным офисом по адресу',
    'цены\n')),
  248),
 (('солнечно лунная теория', ('точной наукой\n',)), 185),
 (('© 2006 2016Bravoavia',
   ('зарегистрированной торговой маркой ® Услуги предоставляемые на сайте предложены Группой компаний lastminute.com group и её партнерами\n',)),
  136),
 (('использовать ваучерBravo КомпенсацияiPhone и Android приложенияПАРТНЕРСТВОРекламаАкции и новостиДанный веб-сайт',
   ('собственностью компании BravoNext SA с головным офисом по адресу Vicolo de’ Calvi 2 6830 Chiasso CH регистрационный номер компании CHE 115.704.228 ИНН CHE 115.704.228\n',)),
  99),
 (('сбор',
   ('дополнительной платой', 'за использование нашей службы бронирования\n')),
  97),
 (('Vicolo de’ Calvi 2 6830 Chiasso CH ре

In [437]:
d['amod obj']

{'бесплатную услугу',
 'большие номера',
 'большой выбор',
 'взаимовыгодное сотрудничество',
 'дизайнерские номера',
 'инжиниринговые услуги',
 'канадские дома',
 'некурящие номера',
 'оригинальные детали',
 'просторные номера',
 'различные номера',
 'смежные номера',
 'специализированные видеокодеки',
 'такие уд',
 'такие удобст',
 'такие удобства',
 'такие фотоуслуги',
 'цветные смесители',
 'экскурсионное бюро.',
 'элегантные номера'}

In [422]:
Counter(seqs).most_common()

[('amod obl', 160),
 ('amod obj', 147),
 ('obl', 63),
 ('obj', 40),
 ('case obl', 10),
 ('amod obj nmod', 7),
 ('case amod obl', 4),
 ('obl nmod', 3),
 ('obj case nummod', 3),
 ('amod obj nmod cc conj nmod', 3),
 ('amod obj amod nmod', 3),
 ('amod amod obj', 3),
 ('obl case', 2),
 ('amod obj nmod cc conj amod obj case nummod cc conj amod obl amod conj nmod cc amod conj nmod',
  2),
 ('obj nmod', 2),
 ('amod obj cc conj', 2),
 ('amod obj case nmod', 2),
 ('case obl nmod amod nmod', 2),
 ('advmod amod obj', 2),
 ('amod obj nmod nmod cc conj', 2),
 ('obj nmod case nmod', 2),
 ('nummod obl', 2),
 ('obj cc conj', 2),
 ('amod obj cc conj nsubj cc parataxis nsubj acl:relcl xcomp case flat:foreign flat:foreign case obl nsubj aux conj obj cc advmod',
  2),
 ('amod obl amod cc conj nmod', 2),
 ('amod obj nmod case amod nmod', 2),
 ('obj nmod amod nmod', 2),
 ('obj nmod case nmod nmod cc case advmod conj cc conj', 1),
 ('obj case nmod case amod nmod advmod parataxis cc conj nmod cc conj', 1),
 ('

In [439]:
c.most_common(100)

[('является', 141080),
 ('нет', 93128),
 ('может', 73867),
 ('имеет', 70084),
 ('можете', 69585),
 ('стал', 57634),
 ('находится', 55386),
 ('являются', 46268),
 ('предлагает', 40765),
 ('нравится', 39737),
 ('стоит', 38130),
 ('получил', 35728),
 ('имеют', 35570),
 ('могут', 33677),
 ('посмотрите', 32962),
 ('составляет', 31540),
 ('стала', 30845),
 ('работает', 29751),
 ('найдете', 26897),
 ('происходит', 24595),
 ('стали', 24544),
 ('следует', 21883),
 ('становится', 20577),
 ('идет', 20360),
 ('удалось', 20183),
 ('делает', 20075),
 ('используется', 20071),
 ('состоит', 20069),
 ('получают', 19788),
 ('помогаем', 19785),
 ('придется', 19720),
 ('находятся', 19619),
 ('входит', 19043),
 ('зависит', 18194),
 ('представляет', 17987),
 ('относится', 17930),
 ('предлагают', 17814),
 ('стало', 16920),
 ('дает', 16576),
 ('говорит', 16547),
 ('станет', 16491),
 ('получила', 16294),
 ('сможете', 16263),
 ('получили', 15929),
 ('включает', 15445),
 ('проходит', 15006),
 ('требуется', 14615)

In [59]:
[x[0] for x in preds['родился'].most_common() if x[0].istitle() and len(x[0]) > 3]

['Мейер Залик',
 'Гвоздев',
 'Одегбами',
 'Джу Хёк',
 'Раманаускас',
 'Яков Тургенев',
 'Гермоген Титович',
 'Стотт',
 'Пузович',
 'А. Анисимов',
 'Ольмедо',
 'Восточная Фризия .',
 'Добрый',
 'Терещенко',
 'Флориан Федорович',
 'Аксельссон',
 'Шмарово',
 'Марьянович',
 'Бочаров',
 'Раймонд Сапорта',
 'Лисица',
 'Аристон',
 'Еникеев',
 'Алексанян',
 'Йованович',
 'Фернандес',
 'Фольман',
 'Бастианини',
 'Прайс',
 'Икинс',
 'Квайе',
 'Чарльз Крамп',
 "Л'Акуила ,",
 'Сантистебан-Ксикес',
 'Мандро',
 '( 1866По',
 'Иенеми',
 'Ильин',
 'Секонди- Такоради .',
 'Миркович',
 'Людвиг Гшоссманн',
 'И. А.',
 'Винья Стеллути',
 'Мохамед',
 'Джереми ,',
 'Зайцев',
 'Джурич',
 'Авраам Цви Идельсон',
 'Волдемар Валдманис —',
 'Аллахвердиев',
 'Мартин Савала Ломбарди',
 'Лазарь Гейгер',
 'Конти',
 'Канада',
 'Марк Алданов',
 'Архипп',
 'Украина .',
 'Сальминен',
 'Раймундо Сапорта',
 'Шень',
 '( Драйсон',
 'Аризанов',
 'Кеннеди',
 'Обадов',
 'Милка Сингх',
 'Сироджиддин Джалолиддиновича',
 'Аристион',

In [60]:
[x for x in sub_obj['родился'].most_common() if x[0][0].istitle()]

[(('Одегбами', ('в нигерийском городе Ибадан .\n',)), 1),
 (('Людвиг Гшоссманн',
   ('Согласно «Общему лексикону художников всех времен и народов» ( Allgemeines Künstlerlexikon : Die bildenden Künstler aller Zeiten und Völker ) Günter Meißner : Allgemeines Künstlerlexikon : Die bildenden Künstler aller Zeiten und Völker',
    'в Мюнхене .\n')),
  1),
 (('Флориан Федорович', ('Флориан Федорович\n',)), 1),
 (('Раймундо Сапорта', ('в 1926 году', 'в Париже ,\n')), 1),
 (('Саут-Йоркшир .', ('Гарри Магуайр', 'в Шеффилде ,\n')), 1),
 (('Милка Сингх', ('Согласно записям в Пакистане\n',)), 1),
 (('Аристион', ('Согласно этой книге', 'в семье язычников .', 'во II веке\n')),
  1),
 (('Аризанов', ('в селе Иловица ( община Босилово ) .\n',)), 1),
 (('Раймонд Сапорта', ('в Стамбуле .', 'согласно которому\n')), 1),
 (('Чарльз Крамп', ('Хью', 'в Рочестере ,\n')), 1),
 (('Архипп', ('в городе Иераполе ,', 'во Фригии ,\n')), 1),
 (('А. Анисимов',
   ('Л.',
    'в селе Ракитное ( сейчас — Гнаровское ) , Во

In [1]:
import requests

In [2]:
yarn = requests.get('http://russianword.net/yarn.xml')

In [5]:
f = open('yarn.xml', 'wb')
f.write(yarn.content)
f.close()